In [1]:
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [58]:
train_features = pd.read_csv('/content/drive/My Drive/moa/train_features.csv')
train_targets_scored = pd.read_csv('/content/drive/My Drive/moa/train_targets_scored.csv')
sample_submission = pd.read_csv('/content/drive/My Drive/moa/sample_submission.csv')
test_features = pd.read_csv('/content/drive/My Drive/moa/test_features.csv')

In [59]:
train_features

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,g-7,g-8,g-9,g-10,g-11,g-12,g-13,g-14,g-15,g-16,g-17,g-18,g-19,g-20,g-21,g-22,g-23,g-24,g-25,g-26,g-27,g-28,g-29,g-30,g-31,g-32,g-33,g-34,g-35,...,c-60,c-61,c-62,c-63,c-64,c-65,c-66,c-67,c-68,c-69,c-70,c-71,c-72,c-73,c-74,c-75,c-76,c-77,c-78,c-79,c-80,c-81,c-82,c-83,c-84,c-85,c-86,c-87,c-88,c-89,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_000644bb2,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,-0.0326,0.5548,-0.0921,1.1830,0.1530,0.5574,-0.4015,0.1789,-0.6528,-0.7969,0.6342,0.1778,-0.3694,-0.5688,-1.1360,-1.1880,0.6940,0.4393,0.2664,0.1907,0.1628,-0.2853,0.5819,0.2934,-0.5584,-0.0916,-0.3010,-0.1537,0.2198,...,0.4805,0.4965,0.3680,0.8427,0.1042,0.1403,0.1758,1.2570,-0.5979,1.2250,-0.0553,0.7351,0.5810,0.9590,0.2427,0.0495,0.4141,0.8432,0.6162,-0.7318,1.2120,0.6362,-0.4427,0.1288,1.4840,0.1799,0.5367,-0.1111,-1.0120,0.6685,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,id_000779bfc,trt_cp,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,0.2341,0.3372,-0.4047,0.8507,-1.1520,-0.4201,-0.0958,0.4590,0.0803,0.2250,0.5293,0.2839,-0.3494,0.2883,0.9449,-0.1646,-0.2657,-0.3372,0.3135,-0.4316,0.4773,0.2075,-0.4216,-0.1161,-0.0499,-0.2627,0.9959,-0.2483,0.2655,-0.2102,...,0.4083,0.0319,0.3905,0.7099,0.2912,0.4151,-0.2840,-0.3104,-0.6373,0.2887,-0.0765,0.2539,0.4443,0.5932,0.2031,0.7639,0.5499,-0.3322,-0.0977,0.4329,-0.2782,0.7827,0.5934,0.3402,0.1499,0.4420,0.9366,0.8193,-0.4236,0.3192,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
2,id_000a6266a,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,0.1715,0.2155,0.0065,1.2300,-0.4797,-0.5631,-0.0366,-1.8300,0.6057,-0.3278,0.6042,-0.3075,-0.1147,-0.0570,-0.0799,-0.8181,-1.5320,0.2307,0.4901,0.4780,-1.3970,4.6240,-0.0437,1.2870,-1.8530,0.6069,0.4290,0.1783,0.0018,-1.1800,...,-0.5477,-0.7576,-0.0444,0.1894,-0.0014,-2.3640,-0.4682,0.1210,-0.5177,-0.0604,0.1682,-0.4436,0.4963,0.1363,0.3335,0.9760,-0.0427,-0.1235,0.0959,0.0690,-0.9416,-0.7548,-0.1109,-0.6272,0.3019,0.1172,0.1093,-0.3113,0.3019,-0.0873,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
3,id_0015fd391,trt_cp,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,-1.9590,0.1792,-0.1321,-1.0600,-0.8269,-0.3584,-0.8511,-0.5844,-2.5690,0.8183,-0.0532,-0.8554,0.1160,-2.3520,2.1200,-1.1580,-0.7191,-0.8004,-1.4670,-0.0107,-0.8995,0.2406,-0.2479,-1.0890,-0.7575,0.0881,-2.7370,0.8745,0.5787,-1.6740,...,-2.1220,-0.3752,-2.3820,-3.7350,-2.9740,-1.4930,-1.6600,-3.1660,0.2816,-0.2990,-1.1870,-0.5044,-1.7750,-1.6120,-0.9215,-1.0810,-3.0520,-3.4470,-2.7740,-1.8460,-0.5568,-3.3960,-2.9510,-1.1550,-3.2620,-1.5390,-2.4600,-0.9417,-1.5550,0.2431,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
4,id_001626bd3,trt_cp,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,-0.2800,-0.1498,-0.8789,0.8630,-0.2219,-0.5121,-0.9577,1.1750,0.2042,0.1970,0.1244,-1.7090,-0.3543,-0.5160,-0.3330,-0.2685,0.7649,0.2057,1.3720,0.6835,0.8056,-0.3754,-1.2090,0.2965,-0.0712,0.6389,0.6674,-0.0783,1.1740,-0.7110,...,-0.2274,0.3215,0.1535,-0.4640,-0.5943,0.3973,0.1500,0.5178,0.5159,0.6091,0.1813,-0.4249,0.7832,0.6529,0.5648,0.4817,0.0587,0.5303,0.6376,-0.3966,-1.4950,-0.9625,-0.0541,0.6273,0.4563,0.0698,0.8134,0.1924,0.6054,-0.1824,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23809,id_fffb1ceed,trt_cp,24,D2,0.1394,-0.0636,-0.1112,-0.5080,-0.4713,0.7201,0.5773,0.3055,-0.4726,0.1269,0.2531,0.1730,-0.4532,-1.0790,0.2474,-0.4550,0.3588,0.1600,-0.7362,-0.1103,0.8550,-0.4139,0.5541,0.2310,-0.5573,-0.4397,-0.9260,-0.2424,-0.6686,0.2326,0.

In [60]:
GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]

In [61]:
train_features["cp_type"].value_counts()
# Can we drop cp_type column? ctl_vehicle is 8% from total.

trt_cp         21948
ctl_vehicle     1866
Name: cp_type, dtype: int64

In [62]:
train = train_features.merge(train_targets_scored, on='sig_id')
test = test_features
# If we choose to drop train_features[train['cp_type']=='ctl_vehicle'], uncomment.
# train = train[train['cp_type']!='ctl_vehicle'].reset_index(drop=True)
# test = test_features[test_features['cp_type']!='ctl_vehicle'].reset_index(drop=True)

target = train[train_targets_scored.columns]
train = train.drop('cp_type', axis=1)               # train["cp_type"].unique() = 'trt_cp'. We cant pass cp_type without encode.
train = train.drop('sig_id', axis=1)

# target                      # 23814 rows × 207 columns. # Its actually the same as train_targets_scored, if we didnt preprocess anythig.

In [63]:
class MoADataset:
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
        
    def __len__(self):              # len() will use the __len__ method if present to get your object for its length.  
        return (self.features.shape[0])
    
    def __getitem__(self, idx):     # docs: https://docs.python.org/3/reference/datamodel.html#object.__getitem__. In this case returns a dict.
        dct = { 
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float),       # ex: np_array[0, :] -> [1,2]
            'y' : torch.tensor(self.targets[idx, :], dtype=torch.float)            
        }
        return dct
    
class TestDataset:
    def __init__(self, features):
        self.features = features
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float)
        }
        return dct
    

In [64]:
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0
    
    for data in dataloader:
        optimizer.zero_grad()
        inputs, targets = data['x'].to(device), data['y'].to(device)        # Asks for the value of "x" and "y" keys.
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    
    return final_loss   

In [65]:
# process_data(data) uses get_dummies() to create cp_time: 24, 48, 72. cp_dose: D1, D2
def process_data(data):
    data = pd.get_dummies(data, columns=['cp_time','cp_dose'])              
   
    return data

In [66]:
# Simply target without id.
target_cols = target.drop('sig_id', axis=1).columns.tolist()
# We use this comprehension to take into account the dummies created by our process_data()
feature_cols = [col for col in process_data(train).columns if col not in target_cols]

# feature_cols = [col for col in feature_cols if c not in ['kfold','sig_id']]


In [67]:
# HyperParameters
DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
# EPOCHS = 25
EPOCHS = 2
BATCH_SIZE = 128
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-5
NFOLDS = 5
EARLY_STOPPING_STEPS = 10
EARLY_STOP = False

num_features=len(feature_cols)
num_targets=len(target_cols)
hidden_size=1024

In [68]:
class Model(nn.Module):
    def __init__(self, num_features, num_targets, hidden_size):
        super(Model, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dropout1 = nn.Dropout(0.2)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size))
        
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(0.5)
        self.dense2 = nn.utils.weight_norm(nn.Linear(hidden_size, hidden_size))
        
        self.batch_norm3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(0.5)
        self.dense3 = nn.utils.weight_norm(nn.Linear(hidden_size, num_targets))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.relu(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)
        
        return x

In [69]:
train_df = process_data(train)
x_train, y_train  = train_df[feature_cols].values, train_df[target_cols].values
train_dataset = MoADataset(x_train, y_train)

In [70]:
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
# It has a len of 187. 187 * 128(BATCH_SIZE) = 23936. Contains the 23814 rows of the train_df.

In [71]:
model = Model(
    num_features=num_features,
    num_targets=num_targets,
    hidden_size=hidden_size,
)

model.to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3,
                                            max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(trainloader))
loss_fn = nn.BCEWithLogitsLoss()
early_stopping_steps = EARLY_STOPPING_STEPS
early_step = 0

oof = np.zeros((len(train), target.iloc[:, 1:].shape[1]))
best_loss = np.inf                  # Represents a positive infinite

for epoch in range(EPOCHS):
    train_loss = train_fn(model, optimizer, scheduler, loss_fn, trainloader, DEVICE)
    print(f"EPOCH: {epoch}, train_loss: {train_loss}")
    # valid_loss, valid_preds = valid_fn(model, loss_fn, validloader, DEVICE)
    # print(f"EPOCH: {epoch}, valid_loss: {valid_loss}")
    
    # if valid_loss < best_loss:
        
    #     best_loss = valid_loss
    #     oof[val_idx] = valid_preds
    #     torch.save(model.state_dict(), f"FOLD{fold}_.pth")
    
    # elif(EARLY_STOP == True):
        
    #     early_step += 1
    #     if (early_step >= early_stopping_steps):
    #         break

EPOCH: 0, train_loss: 0.12769493572214707
EPOCH: 1, train_loss: 0.01842740702537451


In [72]:
test_df = process_data(test)
x_test  = test_df[feature_cols].values
test_dataset = TestDataset(x_test)
testloader = torch.utils.data.DataLoader(test_dataset)

def inference_fn(model, dataloader, device):
    model.eval()
    preds = []
    
    for data in dataloader:
        inputs = data['x'].to(device)

        with torch.no_grad():
            outputs = model(inputs)
        
        preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    preds = np.concatenate(preds)
    
    return preds

predictions = np.zeros((len(test_df), target.iloc[:, 1:].shape[1]))
predictions = inference_fn(model, testloader, DEVICE)

In [73]:
sample_submission = pd.read_csv('/content/drive/My Drive/moa/sample_submission.csv')

In [74]:
y = pd.DataFrame(data=predictions)
y.columns = target_cols
y

,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,autotaxin_inhibitor,...,protein_synthesis_inhibitor,protein_tyrosine_kinase_inhibitor,radiopaque_medium,raf_inhibitor,ras_gtpase_inhibitor,retinoid_receptor_agonist,retinoid_receptor_antagonist,rho_associated_kinase_inhibitor,ribonucleoside_reductase_inhibitor,rna_polymerase_inhibitor,serotonin_receptor_agonist,serotonin_receptor_antagonist,serotonin_reuptake_inhibitor,sigma_receptor_agonist,sigma_receptor_antagonist,smoothened_receptor_antagonist,sodium_channel_inhibitor,sphingosine_receptor_agonist,src_inhibitor,steroid,syk_inhibitor,tachykinin_antagonist,tgf-beta_receptor_inhibitor,thrombin_inhibitor,thymidylate_synthase_inhibitor,tlr_agonist,tlr_antagonist,tnf_inhibitor,topoisomerase_inhibitor,transient_receptor_potential_channel_antagonist,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,0.001244,0.001412,0.001188,0.012443,0.018641,0.004399,0.002702,0.006495,0.001322,0.017687,0.024875,0.002907,0.000780,0.001748,0.000832,0.000742,0.002422,0.004708,0.006008,0.001483,0.002615,0.003082,0.000978,0.001493,0.001073,0.001048,0.000822,0.001202,0.003514,0.002083,0.001598,0.003154,0.001841,0.000910,0.000607,0.000749,0.003867,0.001081,0.003950,0.000680,...,0.002530,0.001036,0.003115,0.000864,0.000934,0.004054,0.000772,0.001935,0.001180,0.001084,0.011567,0.020401,0.002512,0.001563,0.002133,0.000885,0.016942,0.001202,0.002456,0.000620,0.001020,0.003958,0.001346,0.001732,0.001471,0.001483,0.000850,0.001488,0.003106,0.001054,0.000744,0.001259,0.003384,0.004990,0.003127,0.000859,0.007784,0.001376,0.001639,0.001683
1,0.000841,0.001325,0.001895,0.007773,0.011222,0.004629,0.002277,0.006183,0.001403,0.012652,0.017430,0.005458,0.001029,0.003062,0.001223,0.000951,0.002163,0.004524,0.006293,0.002728,0.001873,0.002069,0.001086,0.002517,0.001354,0.001495,0.001010,0.002928,0.003915,0.002383,0.001336,0.003669,0.002117,0.001331,0.000874,0.001253,0.006856,0.001961,0.010240,0.000914,...,0.004755,0.001859,0.002319,0.003193,0.001300,0.004678,0.001189,0.002099,0.003509,0.001607,0.013584,0.030166,0.002053,0.001591,0.002969,0.001091,0.012386,0.001457,0.006194,0.001012,0.001577,0.002627,0.001279,0.001652,0.002014,0.001487,0.001043,0.003631,0.006014,0.001310,0.001230,0.001959,0.002501,0.007483,0.006958,0.001243,0.012046,0.001301,0.002550,0.001712
2,0.000574,0.000433,0.001147,0.007586,0.008681,0.001889,0.001840,0.002156,0.000504,0.007975,0.012891,0.000682,0.000412,0.000979,0.000680,0.000719,0.001027,0.002739,0.001637,0.001396,0.000971,0.003096,0.000382,0.001489,0.000470,0.000432,0.000492,0.000870,0.002421,0.001404,0.000772,0.001956,0.001943,0.000432,0.000453,0.000406,0.002033,0.000463,0.000837,0.000506,...,0.002903,0.000773,0.001869,0.001548,0.000600,0.001590,0.000530,0.000958,0.001233,0.000899,0.008019,0.010282,0.001271,0.000944,0.000698,0.001131,0.006905,0.001523,0.000962,0.000639,0.000504,0.001202,0.001225,0.000470,0.001226,0.000898,0.000567,0.001446,0.001966,0.000571,0.000665,0.000526,0.001922,0.002542,0.001258,0.000484,0.003090,0.001045,0.001288,0.001394
3,0.0006

In [75]:
sub = sample_submission.drop(columns=target_cols)
frames = [sub, y]
sub = pd.concat(frames, axis=1)

In [76]:
submission = sub.to_csv("submission.csv",index=False)
teste = pd.read_csv("submission.csv")
teste

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,...,protein_synthesis_inhibitor,protein_tyrosine_kinase_inhibitor,radiopaque_medium,raf_inhibitor,ras_gtpase_inhibitor,retinoid_receptor_agonist,retinoid_receptor_antagonist,rho_associated_kinase_inhibitor,ribonucleoside_reductase_inhibitor,rna_polymerase_inhibitor,serotonin_receptor_agonist,serotonin_receptor_antagonist,serotonin_reuptake_inhibitor,sigma_receptor_agonist,sigma_receptor_antagonist,smoothened_receptor_antagonist,sodium_channel_inhibitor,sphingosine_receptor_agonist,src_inhibitor,steroid,syk_inhibitor,tachykinin_antagonist,tgf-beta_receptor_inhibitor,thrombin_inhibitor,thymidylate_synthase_inhibitor,tlr_agonist,tlr_antagonist,tnf_inhibitor,topoisomerase_inhibitor,transient_receptor_potential_channel_antagonist,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_0004d9e33,0.001244,0.001412,0.001188,0.012443,0.018641,0.004399,0.002702,0.006495,0.001322,0.017687,0.024875,0.002907,0.000780,0.001748,0.000832,0.000742,0.002422,0.004708,0.006008,0.001483,0.002615,0.003082,0.000978,0.001493,0.001073,0.001048,0.000822,0.001202,0.003514,0.002083,0.001598,0.003154,0.001841,0.000910,0.000607,0.000749,0.003867,0.001081,0.003950,...,0.002530,0.001036,0.003115,0.000864,0.000934,0.004054,0.000772,0.001935,0.001180,0.001084,0.011567,0.020401,0.002512,0.001563,0.002133,0.000885,0.016942,0.001202,0.002456,0.000620,0.001020,0.003958,0.001346,0.001732,0.001471,0.001483,0.000850,0.001488,0.003106,0.001054,0.000744,0.001259,0.003384,0.004990,0.003127,0.000859,0.007784,0.001376,0.001639,0.001683
1,id_001897cda,0.000841,0.001325,0.001895,0.007773,0.011222,0.004629,0.002277,0.006183,0.001403,0.012652,0.017430,0.005458,0.001029,0.003062,0.001223,0.000951,0.002163,0.004524,0.006293,0.002728,0.001873,0.002069,0.001086,0.002517,0.001354,0.001495,0.001010,0.002928,0.003915,0.002383,0.001336,0.003669,0.002117,0.001331,0.000874,0.001253,0.006856,0.001961,0.010240,...,0.004755,0.001859,0.002319,0.003193,0.001300,0.004678,0.001189,0.002099,0.003509,0.001607,0.013584,0.030166,0.002053,0.001591,0.002969,0.001091,0.012386,0.001457,0.006194,0.001012,0.001577,0.002627,0.001279,0.001652,0.002014,0.001487,0.001043,0.003631,0.006014,0.001310,0.001230,0.001959,0.002501,0.007483,0.006958,0.001243,0.012046,0.001301,0.002550,0.001712
2,id_002429b5b,0.000574,0.000433,0.001147,0.007586,0.008681,0.001889,0.001840,0.002156,0.000504,0.007975,0.012891,0.000682,0.000412,0.000979,0.000680,0.000719,0.001027,0.002739,0.001637,0.001396,0.000971,0.003096,0.000382,0.001489,0.000470,0.000432,0.000492,0.000870,0.002421,0.001404,0.000772,0.001956,0.001943,0.000432,0.000453,0.000406,0.002033,0.000463,0.000837,...,0.002903,0.000773,0.001869,0.001548,0.000600,0.001590,0.000530,0.000958,0.001233,0.000899,0.008019,0.010282,0.001271,0.000944,0.000698,0.001131,0.006905,0.001523,0.000962,0.000639,0.000504,0.001202,0.001225,0.000470,0.001226,0.000898,0.000567,0.001446,0.001966,0.000571,0.000665,0.000526,0.001922,0.002542,0.001258,0.000484,0.003090,0.001045,0.001288,0.001394
3,id_0027